## 12.2 地図を使ったグラフ

In [ ]:
import json
import numpy as np
import pandas as pd

from plotly import graph_objects as go
from plotly import data as pdata
from plotly.graph_objs.layout import Template

# 東京駅の緯度経度
tokyo_lat = 35.681
tokyo_lon = 139.767

# Traceの作成
trace = go.Scattermapbox()  # 地図のみ

# 独自テンプレートを読み込み
with open('custom_white.json') as f:
    custom_white_dict = json.load(f)
    template = Template(custom_white_dict)

# Layoutを作成（open-street-mapスタイル）
layout = go.Layout(
    template=template,
    title='Mapbox sample',
    mapbox={
        'style': 'open-street-map',
        'zoom': 14,
        'center': {
            'lat': tokyo_lat,
            'lon': tokyo_lon
        }
    },
    margin={
        'r': 20,    # 余白右
        't': 30,    # 余白上
        'l': 20,    # 余白左
        'b': 30     # 余白下
    }
)

# Figureを作成
figure = go.Figure(trace, layout)

# figure.write_image('./figure/out_12_2_1.png', width=900, height=450, scale=2)
figure

In [ ]:
# FigureのLayoutを更新（carto-positronスタイル）
figure.update_layout(
    mapbox={'style': 'carto-positron'}
)

# figure.write_image('./figure/out_12_2_2.png', width=900, height=450, scale=2)
figure

In [ ]:
# FigureのLayoutを更新（carto-darkmatterスタイル）
figure.update_layout(
    mapbox={'style': 'carto-darkmatter'}
)

# figure.write_image('./figure/out_12_2_3.png', width=900, height=450, scale=2)
figure

In [ ]:
# CarshareデータセットからDataFrameを読み込み
df = pdata.carshare()

df

In [ ]:
# https://wiki.openstreetmap.org/wiki/Zoom_levels

def get_zoom_level(lon_width:float) -> float:
    """Mapboxの倍率レベルを取得する

    Args:
        lon_width (float): 経度の幅

    Returns:
        float: 倍率レベル
    """
    width_levels = np.array([
        360,	    # 0: whole world
        180,        # 1
        90,         # 2: subcontinental area
        45,         # 3: largest country
        22.5,       # 4
        11.25,      # 5: large African country
        5.625,      # 6: large European country
        2.813,      # 7: small country, US state
        1.406,      # 8
        0.703,      # 9: wide area, large metropolitan area
        0.352,      # 10: metropolitan area
        0.176,      # 11: city
        0.088,      # 12: town, or city district
        0.044,      # 13: village, or suburb
        0.022,      # 14
        0.011,      # 15: small road
        0.005,      # 16: street
        0.003,      # 17: block, park, addresses
        0.001,      # 18: some buildings, trees
        0.0005,     # 19: local highway and crossing details
        0.00025,    # 20: A mid-sized building 
    ])

    diff = width_levels - lon_width
    diff[diff < 0] = np.NaN
    return np.nanargmin(diff)

In [ ]:
lon_width = df['centroid_lon'].max() - df['centroid_lon'].min()     # 経度の幅

lon_width

In [ ]:
zoom_level = get_zoom_level(lon_width)  # 倍率レベル

zoom_level

In [ ]:
# Traceを作成
trace = go.Scattermapbox(
    lat=df['centroid_lat'],     # 緯度に使用する変数
    lon=df['centroid_lon']      # 経度に使用する変数
)   # カーシェア場所のドットマップ

# Layoutを作成
layout = go.Layout(
    template=template,
    title='Carshare dataset',
    mapbox={
        'style': 'carto-positron',
        'zoom': zoom_level,
        'center': {
            'lat': df['centroid_lat'].mean(),
            'lon': df['centroid_lon'].mean()
        }
    },
    margin={
        'r': 20,
        't': 30,
        'l': 20,
        'b': 30
    }
)

# Figureを作成
figure = go.Figure(trace, layout)

# figure.write_image('./figure/out_12_2_4.png', width=900, height=450, scale=2)
figure

In [ ]:
max_size = 32                                       # マーカーの最大サイズ
marker_coeff = max_size / df['car_hours'].max()     # シェア時間が最大のマーカーでmax_sizeになる係数

# Traceを作成
trace = go.Scattermapbox(
    lat=df['centroid_lat'],     # 緯度に使用する変数
    lon=df['centroid_lon'],     # 経度に使用する変数
    marker={
        'color': df['car_hours'],
        'size': df['car_hours'] * marker_coeff,
        'sizemode': 'diameter',     # 直径で表現。面積で表現は'area'
        'opacity': 0.5
    }
)   # カーシェア場所と時間のバブルチャート

# Figureを作成
figure = go.Figure(trace, layout)

# figure.write_image('./figure/out_12_2_5.png', width=900, height=450, scale=2)
figure

In [ ]:
# Traceを作成
trace = go.Densitymapbox(
    lat=df['centroid_lat'],     # 緯度に使用する変数
    lon=df['centroid_lon'],     # 経度に使用する変数
    z=df['car_hours'],          # 分布を表現する変数
    radius=20.                  # 影響範囲
)   # イサリスミックマップ

# Layoutを作成
layout = go.Layout(
    template=template,
    title='Carshare dataset',
    mapbox={
        'style': 'carto-darkmatter',
        'zoom': zoom_level,
        'center': {
            'lat': df['centroid_lat'].mean(),
            'lon': df['centroid_lon'].mean()
        }
    },
    margin={
        'r': 20,
        't': 30,
        'l': 20,
        'b': 30
    }
)

# Figureを作成
figure = go.Figure(trace, layout)

# figure.write_image('./figure/out_12_2_6.png', width=900, height=450, scale=2)
figure

In [ ]:
# GapminderデータセットからDataFrameを作成
df = pdata.gapminder()

df

In [ ]:
# Traceを作成
trace = go.Choropleth(
    locations=df.query("year==1952")['iso_alpha'],  # 領域に使用する変数
    z = df.query("year==1952")['lifeExp'],          # 強度に使用する変数
    locationmode = 'ISO-3',                         # 領域モード（ISO-3）
    colorbar_title = "lifeExp",                     # カラーバータイトル
)   # 1952年の平均余命のコロプレスマップ

# Layoutを作成
layout = go.Layout(
    template=template,
    title='Gapminder dataset',
    margin={
        'r': 20,
        't': 30,
        'l': 20,
        'b': 30
    }
)

# Figureを作成
figure = go.Figure(trace, layout)

# figure.write_image('./figure/out_12_2_7.png', width=900, height=450, scale=2)
figure

In [ ]:
# 都道府県のGeoJsonを読み込み
with open('todofuken.geojson') as file:
    geojson = json.load(file)

In [ ]:
# 都道府県人口のCSVからDataFrameを読み込み
df = pd.read_csv('population.csv')

df.head(10)

In [ ]:
# Traceを作成
trace = go.Choropleth(
    locations=df['地域'],
    z=df['#A011000_総人口【万人】'],
    geojson=geojson,
    featureidkey='properties.N03_001'
)

# Layoutを作成
layout = go.Layout(
    template=template,
    title='Population',
    margin={
        'r': 20,
        't': 30,
        'l': 20,
        'b': 30
    },
    geo={
        'fitbounds': 'locations',
        'visible': False,
    }
)

# Figureを作成
figure = go.Figure(trace, layout)

# figure.write_image('./figure/out_12_2_8.png', width=900, height=450, scale=2)
figure